# Python + SQL

### Using SQL with pandas
---

## Introduction

Have you ever wanted to query a database within a Python program?  You can do that with the good old pandas library.

...and yes. There are *other* ways ;).

## Topics

* pandas and SQL
* why SQL in Python?
* the easiest way to SQL
* more robust SQL
* accessing SQL databases

## Imports

In [3]:
import pandas as pd
from pandasql import sqldf, load_births

## Dataset

In [4]:
births = load_births()

---

## The easiest way to use SQL - `sqldf`

* pandas SQL wrapper
* uses SQLite behind the scenes
* can treat pandas data frames as if they were tables
* good for small workloads
* SQL is somewhat limited (i.e. no right outer join)

In [5]:
print(sqldf('SELECT * FROM births WHERE births > 250000 limit 5;', locals()))

                         date  births
0  1975-01-01 00:00:00.000000  265775
1  1975-03-01 00:00:00.000000  268849
2  1975-05-01 00:00:00.000000  254545
3  1975-06-01 00:00:00.000000  254096
4  1975-07-01 00:00:00.000000  275163


### Come.. Let us try a query that spans multiple lines

In [6]:
q = """
  SELECT
    date(date) as DOB,
    sum(births) as "Total Births"
  FROM
    births
  GROUP BY
    date
    limit 5;
"""

print(sqldf(q, locals()))

          DOB  Total Births
0  1975-01-01        265775
1  1975-02-01        241045
2  1975-03-01        268849
3  1975-04-01        247455
4  1975-05-01        254545


In [22]:
print(pysqldf("""
  SELECT FirstName, LastName, EnglishOccupation as "Job Title"
  FROM df_customer
  ORDER BY LastName, FirstName limit 3
"""))

  FirstName LastName       Job Title
0     Aaron    Adams  Skilled Manual
1      Adam    Adams      Management
2      Alex    Adams  Skilled Manual


### Variable scope switcher function

In [8]:
# use this fn to change locals to globals in one place if you need to
def pysqldf(q):
    return sqldf(q, globals()) # (q, locals())

### How to load your own data (csv) with pandas

In [15]:
df_customer = pd.read_csv('./data/DimCustomer.csv')
df_customer.head(3)

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,Suffix,Gender,EmailAddress,YearlyIncome,TotalChildren,NumberChildrenAtHome,EnglishEducation,SpanishEducation,FrenchEducation,EnglishOccupation,SpanishOccupation,FrenchOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase,CommuteDistance
0,11000,26,AW00011000,NaN,Jon,V,Yang,0,1971-10-06,M,NaN,M,jon24@adventure-works.com,90000.0,2,0,Bachelors,Licenciatura,Bac + 4,Professional,Profesional,Cadre,1,0,3761 N. 14th St,NaN,1 (11) 500 555-0162,2011-01-19,1-2 Miles
1,11001,37,AW00011001,NaN,Eugene,L,Huang,0,1976-05-10,S,NaN,M,eugene10@adventure-works.com,60000.0,3,3,Bachelors,Licenciatura,Bac + 4,Professional,Profesional,Cadre,0,1,2243 W St.,NaN,1 (11) 500 555-0110,2011-01-15,0-1 Miles
2,11002,31,AW00011002,NaN,Ruben,NaN,Torres,0,1971-02-09,M,NaN,M,ruben35@adventure-works.com,60000.0,3,3,Bachelors,Licenciatura,Bac + 4,Professional,Profesional,Cadre,1,1,5844 Linden Land,NaN,1 (11) 500 555-0184,2011-01-07,2-5 Miles


### Set your own index

In [16]:
df_customer.set_index('CustomerKey', inplace=True)
df_customer.head(3)

,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,Suffix,Gender,EmailAddress,YearlyIncome,TotalChildren,NumberChildrenAtHome,EnglishEducation,SpanishEducation,FrenchEducation,EnglishOccupation,SpanishOccupation,FrenchOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase,CommuteDistance
CustomerKey,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11000,26,AW00011000,NaN,Jon,V,Yang,0,1971-10-06,M,NaN,M,jon24@adventure-works.com,90000.0,2,0,Bachelors,Licenciatura,Bac + 4,Professional,Profesional,Cadre,1,0,3761 N. 14th St,NaN,1 (11) 500 555-0162,2011-01-19,1-2 Miles
11001,37,AW00011001,NaN,Eugene,L,Huang,0,1976-05-10,S,NaN,M,eugene10@adventure-works.com,60000.0,3,3,Bachelors,Licenciatura,Bac + 4,Professional,Profesional,Cadre,0,1,2243 W St.,NaN,1 (11) 500 555-0110,2011-01-15,0-1 Miles
11002,31,AW00011002,NaN,Ruben,NaN,Torres,0,1971-02-09,M,NaN,M,ruben35@adventure-works.com,60000.0,3,3,Bachelors,Licenciatura,Bac + 4,Professional,Profesional,Cadre,1,1,5844 Linden Land,NaN,1 (11) 500 555-0184,2011-01-07,2-5 Miles


### Examine the datatypes in your dataframe

In [ ]:
df_internet_sales = pd.read_csv('./data/FactInternetSales.csv')
df_internet_sales.dtypes # returns a Series with the data type of each column

### Join example

In [23]:
q = """
  SELECT *
  FROM df_customer
  left outer join df_internet_sales
    on (df_customer.CustomerKey = df_internet_sales.CustomerKey) limit 3
"""
pysqldf(q)

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,Suffix,Gender,EmailAddress,YearlyIncome,TotalChildren,NumberChildrenAtHome,EnglishEducation,SpanishEducation,FrenchEducation,EnglishOccupation,SpanishOccupation,FrenchOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase,CommuteDistance,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,RevisionNumber,OrderQuantity,UnitPrice,ExtendedAmount,UnitPriceDiscountPct,DiscountAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,CustomerPONumber,OrderDate,DueDate,ShipDate
0,11000,26,AW00011000,None,Jon,V,Yang,0,1971-10-06,M,None,M,jon24@adventure-works.com,90000.0,2,0,Bachelors,Licenciatura,Bac + 4,Professional,Profesional,Cadre,1,0,3761 N. 14th St,None,1 (11) 500 555-0162,2011-01-19,1-2 Miles,214,20130503,20130515,20130510,11000,1,6,9,SO57418,4,1,1,34.99,34.99,0,0,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2013-05-03 00:00:00.000,2013-05-15 00:00:00.000,2013-05-10 00:00:00.000
1,11000,26,AW00011000,None,Jon,V,Yang,0,1971-10-06,M,None,M,jon24@adventure-works.com,90000.0,2,0,Bachelors,Licenciatura,Bac + 4,Professional,Profesional,Cadre,1,0,3761 N. 14th St,None,1 (11) 500 555-0162,2011-01-19,1-2 Miles,344,20110119,20110131,20110126,11000,1,6,9,SO43793,1,1,1,3399.99,3399.99,0,0,1912.1544,1912.1544,3399.99,271.9992,84.9998,None,None,2011-01-19 00:00:00.000,2011-01-31 00:00:00.000,2011-01-26 00:00:00.000
2,11000,26,AW00011000,None,Jon,V,Yang,0,1971-10-06,M,None,M,jon24@adventure-works.com,90000.0,2,0,Bachelors,Licenciatura,Bac + 4,Professional,Profesional,Cadre,1,0,3761 N. 14th St,None,1 (11) 500 555-0162,2011-01-19,1-2 Miles,353,20130118,20130130,20130125,11000,2,6,9,SO51522,1,1,1,2319.99,2319.99,0,0,1265.6195,1265.6195,2319.99,185.5992,57.9998,None,None,2013-01-18 00:00:00.000,2013-01-30 00:00:00.000,2013-01-25 00:00:00.000
